In [78]:
import numpy as np
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector
from qiskit.algorithms import IterativeAmplitudeEstimation
from qiskit_finance.applications.estimation import EuropeanCallPricing
from qiskit_finance.circuit.library import NormalDistribution

In [79]:
# Set the parameters
bounds = np.array([0.0, 7.0])
num_qubits = 3
parameters = [0.367845, 0.128729, 0.908456, 0.621987, 0.485623, 0.793521]
strike_price = 3
payoff_approx = 0.5

In [80]:
# Set up the circuits
distrib = NormalDistribution(num_qubits, mu=1, sigma=1, bounds=bounds)
qcirc = TwoLocal(num_qubits, "ry", "cz", entanglement="circular", reps=1)
theta = qcirc.ordered_parameters
unc_cir = distrib.compose(qcirc)

In [81]:
# Calculate the probabilities and plot the distributions
values = np.array([bounds[0] + (bounds[1] - bounds[0]) * x / (2**num_qubits - 1) for x in range(2**num_qubits)])
uncertainty_model = unc_cir.assign_parameters(dict(zip(theta, parameters)))
amplitudes = Statevector.from_instruction(uncertainty_model).data
y = np.abs(amplitudes) ** 2

In [82]:
# Calculate the payoff function and plot it
x = values
y_strike = np.maximum(0, x - strike_price)
european_call_pricing = EuropeanCallPricing(num_qubits, strike_price=strike_price, rescaling_factor=payoff_approx, bounds=bounds, uncertainty_model=uncertainty_model)

In [83]:
# Estimate the value
epsilon = 0.01
alpha = 0.05
problem = european_call_pricing.to_estimation_problem()
ae = IterativeAmplitudeEstimation(epsilon_target=epsilon, alpha=alpha, sampler=Sampler(run_options={"shots": 100}))
result = ae.estimate(problem)

In [84]:
# Print the results
print("Estimated value:    \t%.4f" % (result.estimation_processed))
conf_int = np.array(result.confidence_interval_processed)
print("Confidence interval:\t[%.4f, %.4f]" % tuple(conf_int))

Estimated value:    	-0.5463
Confidence interval:	[-0.5465, -0.5462]
